## Pulling from a dataset that has county and census tract columns

- then we can isolate for California
- isolate relevant columns
- upload to aws

In [1]:
import os
import sys
import pandas as pd

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
sys.path.append(os.path.expanduser('../../'))

# Adjust display options, helpful for long descriptions within ACS data
pd.set_option('display.max_colwidth', None)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import (
    pull_csv_from_directory, upload_csv_aws
)

In [6]:
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/society_economy/vulnerable_populations/usda/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'foodaccess2019.csv'
Saved DataFrame as 'foodaccess2019_readme.csv'
Saved DataFrame as 'foodaccess2019_variable_names.csv'


In [9]:
food_access = pd.read_csv('foodaccess2019.csv')

In [12]:
food_access.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,1001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,1001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,1001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,1001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0


In [17]:
# isolate relevant columns
tract_county = food_access[['CensusTract', 'County', 'State']]


In [19]:
# isolate California data
california_tracts = tract_county[tract_county['State'] == 'California']


In [20]:
california_tracts

,CensusTract,County,State
3551,6001400100,Alameda County,California
3552,6001400200,Alameda County,California
3553,6001400300,Alameda County,California
3554,6001400400,Alameda County,California
3555,6001400500,Alameda County,California
...,...,...,...
11570,6115040800,Yuba County,California
11571,6115040901,Yuba County,California
11572,6115040902,Yuba County,California
11573,6115041000,Yuba County,California


In [41]:
# get rid of county ending string within county column
california_tracts_county = california_tracts[['CensusTract', 'County']]
california_tracts_county.loc[:, 'County'] = california_tracts_county['County'].str.replace(' County', '')
california_tracts_county.tail(5)

,CensusTract,County
11570,6115040800,Yuba
11571,6115040901,Yuba
11572,6115040902,Yuba
11573,6115041000,Yuba
11574,6115041100,Yuba


In [39]:
# save data as a csv
california_tracts_county.to_csv('ca_tracts_county.csv')

In [40]:
# upload csv to aws
bucket_name = 'ca-climate-index'
file_name = 'ca_tracts_county.csv'
directory = '0_map_data'

upload_csv_aws([file_name], bucket_name, directory)
# Remove final csv files from local directory
os.remove(file_name)

ca_tracts_county.csv uploaded to AWS
